In [ ]:
import statsmodels.api as sm
import numpy as np 

class ARX2(sm.tsa.statespace.MLEModel):
    
    def __init__(self, y_t, x_t):
        exen  = np.c_[y_t, y_t, x_t]   # shaped nobs x 2
        nobs  = len(y_t)
        exen  = np.c_[np.ones(nobs), np.zeros(nobs), x_t]
        self.k_posdef = 2
        self.k_exog = 1
        
        super(ARX2, self).__init__(endog=y_t, exog=x_t, k_posdef=self.k_posdef, k_states=3, initialization='approximate_diffuse')

        self['design']     = exen.T[np.newaxis, :, :]  # shaped 1 x 2(exog+endog) x nobs        
        self['transition'] = np.eye(self.k_states)
        self['selection']  = np.eye(self.k_states, self.k_posdef)

    def update(self, params, **kwargs):                    
        params = super(ARX2, self).update(params, **kwargs)

        self['transition', 0, 0] = params[0]
        self['transition', 1, 1] = params[1]
        self['transition', 2, 2] = params[2]
        
        self['state_cov'] = np.diag(params[3:3+self.k_posdef])
        
        self['obs_intercept', 0, 0] = params[-2]
        self['obs_cov', 0, 0] = params[-1]

    # Specify start parameters and parameter names
    @property
    def start_params(self):
#         return [1,1,1, 0.5,0.5, 0.5,0.5]  # these are very simple
        return [1,1,0, 1,1, 1,1]  # these are very simple